# load data

In [1]:
from utils.opconNosepokeFunctions import *
from utils.supplementaryFunctions import *
from scipy.optimize import minimize
from scipy.stats import entropy
from scipy.stats import ttest_rel
%load_ext autoreload
%autoreload 2
import pandas as pd


import numpy as np
from utils.plotSettings import *
# import pickle 
# with open('L:/4portProb_processed/cleandf.pkl', 'rb') as f:
#     df = pickle.load(f)

sessdf = pd.read_csv('S:/fileTransferFromWindows/sessdf.csv')

window = 5
# sessdf = df
sessdf['rr'] = sessdf.groupby(['animal', 'session'], as_index = False).reward.rolling(window, center=True).mean().reward
bin_size = 50
sessdf['sess_bin'] = sessdf.groupby(['animal', 'task'])['session'].transform(lambda x: pd.cut(x, bins=range(0, x.max() + bin_size, bin_size), labels=False, right=False)+1)
# sessdf.drop(columns = 'Unnamed: 0', inplace = True)
exclude = ['[ 20  20  20 100]', '[0 0 0 0]', '[0]', '[0 0]',
    '[1000   80]', '[30]', '[40]', '[70]']
sessdf = sessdf[~sessdf.rewprobfull.isin(exclude)]
sessdf = sessdf[~(sessdf.animal == 'Raikou')]
# Remove sessions (per animal) that have any duplicated trials (rows)
# Assumes 'animal' and 'session' columns exist in sessdf

def remove_sessions_with_duplicate_trials(df, animal_col='animal', session_col='session'):
    # Find sessions (per animal) with any duplicated trials based on 'trialstart' and 'eptime'
    trial_cols = ['trialstart', 'eptime']
    to_remove = set()
    for (animal, session), group in df.groupby([animal_col, session_col]):
        # If any duplicated trial in this group, mark for removal
        if group.duplicated(subset=trial_cols, keep=False).any():
            # print(f"Session {animal}, {session} has duplicated trials based on {trial_cols}")
            to_remove.add((animal, session))
    # Build mask for rows to keep
    mask = ~(df[[animal_col, session_col]].apply(tuple, axis=1).isin(to_remove))
    return df[mask].copy()

# Apply to sessdf
sessdf = remove_sessions_with_duplicate_trials(sessdf)
an_dict = {
    'unstr'  : ['test05022023', 'Blissey', 'Chikorita', 'Darkrai', 'Eevee', 
                'Goldeen', 'Hoppip', 'Inkay', 'Jirachi', 'Kirlia', 'Mesprit',
                'Nidorina', 'Oddish', 'Phione', 'Quilava', 'Raltz', 'Shinx',
                'Togepi', 'Umbreon', 'Vulpix', 'Xatu', 'Yanma', 'Zacian',
                'Alakazam', 'Bayleef', 'Cresselia', 'Emolga', 'Giratina',
                'Haxorus', 'Ivysaur', 'Jigglypuff', 'Lugia', 'Ninetales', 'Onix',
                'Pichu', 'Quaxly', 'Sableye', 'Torchic', 'Uxie', 'Vanillish',
                'Whismur', 'Xerneas', 'Yamper', 'Zorua'],
    'dls'    : ['Blissey', 'Darkrai', 'Inkay' , 'Mesprit', 'Raltz', 'Zacian', 'Alakazam', 'Quilava', 'Torchic', 'Uxie', 'Vanillish', 'Yamper', 'Zorua'],
    'dms'    : ['Hoppip', 'Kirlia', 'Nidorina', 'Togepi', 'Xatu', 'Yanma', 'Giratina', 'Ivysaur', 'Lugia', 'Whismur', 'Quaxly', 'Sableye'],
    'sham'   : ['test05022023', 'Jirachi', 'Goldeen', 'Phione', 'Umbreon', 'Vulpix', 'Emolga', 'Haxorus', 'Jigglypuff'],
    'ds'     : ['Chikorita', 'Eevee'],
    'str'    : ['test05022023', 'Blissey', 'Chikorita', 'Darkrai', 'Eevee'],
    'oe_drive' : ['Shinx', 'Dratini', 'Bayleef', 'Onix', 'Pichu', 'Xerneas']
}

<Figure size 640x480 with 0 Axes>

# log reg

In [ ]:
# log-reg across trials to predict choice on next trial dependent on choices, outcomes, etc
from statsmodels.